In [1]:
import cv2
import numpy as np
import os
import pandas as pd

In [2]:
def define_range(path):
    ''' Defining the ranges of the first and second pages '''
    # Original image
    shape = cv2.imread(path,0).shape
    # Range of x_0
    x_0_lower = 0
    x_0_upper = shape[1]*(4/10)
    # Range of x_1
    x_1_lower = shape[1]*(4/10) 
    x_1_upper = shape[1]*(7/10)    

    return x_0_lower, x_0_upper, x_1_lower, x_1_upper

In [3]:
def change_x(coord, x_0_lower, x_0_upper, x_1_lower, x_1_upper):  
    ''' Returns the page where the word is
        0 in the first page
        1 in the second page
        -1 if not in the range '''
    
    if coord >= x_0_lower and coord <= x_0_upper:
        return 0
    elif coord >= x_1_lower and coord <= x_1_upper:
        return 1
    else:
        return -1

In [12]:
def find_bounds(text, dict_bounds, mask, x_0_lower, x_0_upper, x_1_lower, x_1_upper):
    ''' Finds the names and bounds in the image 
        '''
    proba = np.array([])
    
    # Defining the portion of the height of the box we want to keep
    ratio_y = int((text['Bottom_Right_Y'] - text['Top_Left_Y'])*WIDTH_BOX)
    # Defining the portion of the width of the box we want to keep
    ratio_x = int((text['Bottom_Right_X'] - text['Top_Left_X'])*HEIGHT_BOX)
    # Going through every pixel of the reduced box
    for y in range(text['Top_Left_Y'] + ratio_y, text['Bottom_Right_Y'] - ratio_y):
        for x in range(text['Top_Left_X'] + ratio_x, text['Bottom_Right_X'] - ratio_x):
            # Find their associated probability of being a name
            proba = np.append(proba, mask[y][x])
    # Finding the mean probability of being a name for all the pixels in the reduced box        
    mean = proba.mean()
    if mean > THRESHOLD_PROBA:
        # Finding the page from which 
        coord_x = change_x(text['Top_Left_X'], x_0_lower, x_0_upper, x_1_lower, x_1_upper)
        if coord_x != -1:
            if coord_x in dict_bounds:
                dict_bounds[coord_x].append((text['Top_Left_Y'], text['Text']))
            else:    
                dict_bounds[coord_x] = [(text['Top_Left_Y'], text['Text'])]

In [13]:
def find_names_one_image(page):
    ''' Returns the names and bounds in one image '''
    
    # Data from segmentation
    segmentation_path = "./data/Antigone/1_Segmentation_results/" + page + ".npy"
    data = np.load(segmentation_path)
    
    dict_bounds = dict()
    # Create x ranges
    x_0_lower, x_0_upper, x_1_lower, x_1_upper = define_range("./data/Antigone/0_Images/" + page + ".jpg")
    # Threshold for name segmentation
    mask = np.where(data[1]>THRESHOLD_NAMES,1,0).astype(np.uint8)
    
    # Load results from OCR
    image_df = pd.read_csv('./data/Antigone/2_OCR_results/annotations_' + page + '.csv', index_col=0)
    
    # Find the names and bounds
    image_df.apply(lambda row: find_bounds(row, dict_bounds, mask, x_0_lower, x_0_upper, x_1_lower, x_1_upper), axis=1)
    
    return dict_bounds
                

In [14]:
THRESHOLD_NAMES = 0.2
WIDTH_BOX = 0.4
HEIGHT_BOX = 0.4
THRESHOLD_PROBA = 0.7

def find_names():
    # Going through all the images
    for filename in os.listdir("./data/Antigone/0_Images"):
        if filename.endswith(".jpg"): 
            file_without_extension = os.path.splitext(filename)[0]
            print(file_without_extension)
            # Find names in the image
            dict_bounds = find_names_one_image(file_without_extension)
            print(dict_bounds)
            continue
        else:
            continue

In [15]:
find_names()

p12
{0: [(76, '30'), (111, 'Lea.'), (159, 'Cre.'), (203, 'Erm.'), (752, 'Cre.')], 1: [(106, 'Lea.'), (124, 'Ale.'), (144, 'Forfe'), (173, 'Eur.'), (240, 'Ant.'), (349, 'Cre.'), (455, 'Eur.'), (520, 'Cre.'), (564, 'Lea.'), (628, 'Ant.'), (672, 'Alc.'), (694, 'Cre.')]}
p13
{0: [(172, 'Io'), (70, '32'), (413, 'Eur.'), (569, 'Ant.'), (612, 'Eur.'), (636, 'Ant.'), (656, 'Eur.'), (679, 'Deggio'), (697, 'Ant.')], 1: [(114, 'Eur.'), (546, 'Ant.'), (721, 'Lea.'), (742, 'Forfe'), (763, 'Alc.'), (786, 'Alib.')]}
p11
{0: [(59, '28'), (170, 'Erm.'), (213, 'Alc.'), (278, 'Erm.'), (297, 'Eur.'), (344, 'Erm.'), (386, 'Alc.'), (474, 'Erm.'), (541, 'Eur.'), (583, 'Alc.'), (607, 'Erm.'), (737, 'Cre.')], 1: [(95, 'Lea.'), (137, 'Alc.'), (181, 'Eur.'), (207, 'Cre.'), (229, 'Erm.'), (250, 'Cre.'), (464, 'Erm.'), (489, 'Cre.'), (553, 'Lea.'), (574, 'Eur.'), (617, 'Alc.'), (770, 'Erm,')]}
p10
{0: [(80, '26'), (256, 'Miferi'), (214, 'Eur.'), (707, 'Fine')], 1: [(288, 'Ale'), (353, 'Eur.'), (612, 'Alc.'), (653,

{0: [(89, 'Cre.'), (133, 'altrove.'), (220, 'Eur.'), (245, 'Erm.'), (267, 'Lea.'), (311, 'Cre.'), (572, 'Eur.'), (595, 'Cre.'), (727, 'Eur.'), (750, 'Cre.')], 1: [(354, 'Ant.'), (528, 'Cre.'), (636, 'Eur.'), (697, 'Cre.'), (721, 'Eur.')]}
